## Development Log

## AI usage

## Tasks

### Library imports

In [ ]:
import requests
import pandas as pd
import numpy as np
import json

import openmeteo_requests
import requests_cache
from retry_requests import retry

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.fft import dct, idct
import scipy.stats as stats
from scipy.signal import stft
from sklearn.neighbors import LocalOutlierFactor
from statsmodels.tsa.seasonal import STL

### 1. Create the dataframe for the five cities in Norway

In [ ]:
basic_data = {
    "city":["Oslo", "Kristiansand", "Trondheim", "Tromsø", "Bergen"],
    "price_area_code":["NO1", "NO2", "NO3", "NO4", "NO5"],
    "latitude": [59.9127, 58.1467, 63.4305, 69.6489, 60.393],
    "longitude": [10.7461, 7.9956, 10.3951, 18.9551, 5.3242]
}

basic_info = pd.DataFrame(basic_data)

### 2. Create a function to load data using API

In [ ]:
def load_data_fromAPI(longitude, latitude, selected_year):
	# Setup the Open-Meteo API client with cache and retry on error
	cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
	retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
	openmeteo = openmeteo_requests.Client(session = retry_session)

	# Make sure all required weather variables are listed here
	# The order of variables in hourly or daily is important to assign them correctly below
	url = "https://archive-api.open-meteo.com/v1/archive"
	params = {
		"latitude": latitude,
		"longitude": longitude,
        "start_date": f"{selected_year}-01-01",
        "end_date": f"{selected_year}-12-31",
		"hourly": ["temperature_2m", "wind_speed_10m", "wind_gusts_10m", "wind_direction_10m", "precipitation"],
		"models": "era5",
		"timezone": "auto",
	}
	responses = openmeteo.weather_api(url, params=params)

	# Process first location. Add a for-loop for multiple locations or weather models
	response = responses[0]
	print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
	print(f"Date_range: {params['start_date']} - {params['end_date']}")
	print(f"Variables: {params['hourly']}")
	#print(f"Elevation: {response.Elevation()} m asl")
	#print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

	# Process hourly data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
	hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
	hourly_wind_gusts_10m = hourly.Variables(2).ValuesAsNumpy()
	hourly_wind_direction_10m = hourly.Variables(3).ValuesAsNumpy()
	hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}

	hourly_data["temperature_2m"] = hourly_temperature_2m
	hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
	hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
	hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
	hourly_data["precipitation"] = hourly_precipitation

	hourly_dataframe = pd.DataFrame(data = hourly_data)
	# Change the time zone to Europe/Oslo
	hourly_dataframe["date"] = hourly_dataframe["date"].dt.tz_convert("Europe/Oslo")
	print(f"Sucessfully load the data")
	
	return hourly_dataframe

In [ ]:
# here, we use Bergen as a example to test our function
selected_year = 2019
selected_city = 'Bergen'
latitude = basic_info[basic_info["city"] == selected_city]['latitude']
longitude = basic_info[basic_info["city"] == selected_city]['longitude']
hourly_dataframe = load_data_fromAPI(longitude,latitude,selected_year)

Coordinates: 60.5°N 5.25°E
Date_range: 2019-01-01 - 2019-12-31
Variables: ['temperature_2m', 'wind_speed_10m', 'wind_gusts_10m', 'wind_direction_10m', 'precipitation']
Sucessfully load the data


Briefly explore the dataset

In [9]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype                      
---  ------              --------------  -----                      
 0   date                8760 non-null   datetime64[ns, Europe/Oslo]
 1   temperature_2m      8760 non-null   float32                    
 2   wind_speed_10m      8760 non-null   float32                    
 3   wind_gusts_10m      8760 non-null   float32                    
 4   wind_direction_10m  8760 non-null   float32                    
 5   precipitation       8760 non-null   float32                    
dtypes: datetime64[ns, Europe/Oslo](1), float32(5)
memory usage: 239.7 KB


In [10]:
hourly_dataframe.describe()

,temperature_2m,wind_speed_10m,wind_gusts_10m,wind_direction_10m,precipitation
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,8.846621,14.565521,29.391081,186.588104,0.254053
std,5.089198,8.156624,15.241786,101.865685,0.541051
min,-5.450000,0.000000,4.680000,0.674022,0.000000
25%,4.750000,8.373386,17.639999,107.915705,0.000000
50%,8.050000,12.849528,26.280001,164.664124,0.000000
75%,12.700000,19.657119,38.519997,286.471169,0.300000
max,29.350000,55.753529,119.879997,360.000000,7.300000


In [11]:
hourly_dataframe.head()

,date,temperature_2m,wind_speed_10m,wind_gusts_10m,wind_direction_10m,precipitation
0,2019-01-01 00:00:00+01:00,6.70,42.671768,81.720001,260.776154,0.4
1,2019-01-01 01:00:00+01:00,6.55,47.959782,87.839996,277.765076,0.5
2,2019-01-01 02:00:00+01:00,6.80,48.621330,80.279999,296.375275,0.9
3,2019-01-01 03:00:00+01:00,6.85,52.638840,85.320000,310.006195,0.7
4,2019-01-01 04:00:00+01:00,6.55,55.753529,98.639999,314.215271,0.6


In [12]:
hourly_dataframe.tail()

,date,temperature_2m,wind_speed_10m,wind_gusts_10m,wind_direction_10m,precipitation
8755,2019-12-31 19:00:00+01:00,6.30,15.137133,27.719999,205.346130,0.5
8756,2019-12-31 20:00:00+01:00,6.70,15.990646,30.239998,211.184921,0.5
8757,2019-12-31 21:00:00+01:00,7.10,17.414474,31.680000,209.744797,0.4
8758,2019-12-31 22:00:00+01:00,7.35,19.513195,36.000000,209.875900,0.3
8759,2019-12-31 23:00:00+01:00,7.75,20.056877,38.519997,201.037582,0.2


### 3. Outliers and anomalies:

#### Plot the temperature as a function of time.
Performs high-pass filtering with DCT to remove seasonal effects and detects outliers using robust MAD-based SPC boundaries. Returns an interactive Plotly plot.

In [165]:
def plot_outlier_detection_dct(hourly_dataframe,selected_variable: str, W_filter: float = 1/(10*24), coef_k: float = 3):
    selected_data = hourly_dataframe[selected_variable]
    N = hourly_dataframe.shape[0]
    dt = 1
    W = np.linspace(0, 1/(2*dt), N) # cycles/hour
    # Discrete Cosine transform
    fourier_signal = dct(selected_data.values, type=1, norm="forward")
    filtered_fourier_signal = fourier_signal.copy()
    filtered_fourier_signal[(W < W_filter)] = 0 # high-pass filter
    satv = idct(filtered_fourier_signal, type=1, norm="forward") 

    # Median absolute deviation
    coef_k = coef_k
    trimmed_means = stats.trim_mean(satv, 0.05)
    mad =stats.median_abs_deviation(satv)
    sd = mad * 1.4826
    # Find the boundaries
    upper_boundary = trimmed_means + coef_k * sd
    lower_boundary = trimmed_means - coef_k * sd
    # Detect the outliers
    outliers = satv[np.abs(satv-trimmed_means) > coef_k*sd]
    outliers_index = np.where(np.abs(satv-trimmed_means) > coef_k*sd)[0]

    # Plot data and add lines for +/- 3 SD and the identified outliers
    fig = go.Figure()
    fig.add_hline(y=upper_boundary,line_color='red',line_dash='dash',annotation_text=f'{coef_k}*SD (Upper)',annotation_position='top right')
    fig.add_hline(y=lower_boundary,line_color='red',line_dash='dash',annotation_text=f'{coef_k}*SD (Lower)',annotation_position='top right')
    fig.add_trace(go.Scatter(x=hourly_dataframe['date'], y=selected_data, mode='markers', marker=dict(color='blue'), name='normal'))
    fig.add_trace(go.Scatter(x=hourly_dataframe.loc[outliers_index,'date'], y=outliers, mode='markers', marker=dict(color='orange'), name='outlier'))
    fig.update_layout(title=f'The distribution of {selected_variable} with boundaries and outliers', xaxis_title='Time (hourly)', yaxis_title='Values')

    summary = {
        "variable": selected_variable,
        "num_sample": N,
        "Sigma Multiplier (k)":coef_k,
        "High-pass Filter W_cutoff":W_filter,
        "upper_boundary": upper_boundary,
        "lower_boundary": lower_boundary,
        "num_outliers": len(outliers_index),
        "ratio_outlier": round(len(outliers_index) / N * 100, 2)
    }
    return fig,summary

In [166]:
fig,dct_summary = plot_outlier_detection_dct(hourly_dataframe,"temperature_2m")
for k, v in dct_summary.items():
    print(f"{k}: {v}")

variable: temperature_2m
num_sample: 8760
Sigma Multiplier (k): 3
High-pass Filter W_cutoff: 0.004166666666666667
upper_boundary: 5.287843227386475
lower_boundary: -5.313112735748291
num_outliers: 29
ratio_outlier: 0.33


In [164]:
fig.show()

#### Plot the precipitation as a function of time.

Indicate anomalies according to the Local Outlier Factor method.
- Let the proportion of outliers be a parameter defaulting to 1%.
- Wrap this in a function that returns the plot and relevant summaries of the outliers, and test the function.

In [97]:
def plot_outlier_detection_lof(hourly_dataframe, selected_variable: str, contamination: float = 0.01, n_neighbors: int = 20):
    selected_data = hourly_dataframe[[selected_variable]].copy()
    selected_data['index'] = selected_data.index

    lof = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
    pred_labels = lof.fit_predict(selected_data)

    # Separate normal and outliers
    outlier_mask = pred_labels == -1
    normal_mask = pred_labels == 1


    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hourly_dataframe.loc[normal_mask, 'date'],y=selected_data.loc[normal_mask, selected_variable], mode='markers', marker=dict(color='blue'), name='normal'))
    fig.add_trace(go.Scatter(x=hourly_dataframe.loc[outlier_mask, 'date'],y=selected_data.loc[outlier_mask, selected_variable], mode='markers', marker=dict(color='orange'), name='outlier'))
    fig.update_layout(title=f'The distribution of {selected_variable} with outliers', xaxis_title='Time (hourly)', yaxis_title='Values')

    return fig

In [98]:
fig = plot_outlier_detection_lof(hourly_dataframe,"precipitation")
fig.show()

### 4.Seasonal-Trend decomposition using LOESS (STL)

Perform LOESS on the production data from elbub (downloaded in part 2 of the project) and plot its decomposition.
- Let the electricity price area, production group, period length, seasonal smoother, trend smoother and robust (true/false) be parameters, and give each of them sensible defaults.
- Wrap this in a function that returns the plot, and test the function.

In [ ]:
with open("config_local.json") as f:
        config = json.load(f)

mongo_uri = config["mongo_uri"]

# Create a new client and connect to the server
client = MongoClient(mongo_uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

# connect to the Mongodb
db = client["elhub_db"] 
collection = db["production_data"] 

Pinged your deployment. You successfully connected to MongoDB!


In [117]:
# check the number of data
count = collection.count_documents({})
print(f"Document count: {count:,}")

Document count: 215,033


In [ ]:
# process the starttime column and sort the values by starttime
cursor = collection.find({}, {"_id": 0}) # remove the id 
df_production = pd.DataFrame(list(cursor))
df_production.head()

,pricearea,productiongroup,starttime,quantitykwh
0,NO5,thermal,2021-01-01 06:00:00,77913.0
1,NO5,thermal,2021-01-01 08:00:00,78222.0
2,NO5,thermal,2021-01-01 10:00:00,78141.0
3,NO5,thermal,2021-01-01 11:00:00,78399.0
4,NO5,thermal,2021-01-01 15:00:00,78157.0


In [136]:
df_production['starttime'] = pd.to_datetime(df_production['starttime']) 
df_production = df_production.sort_values("starttime")

In [ ]:
def plot_stl_decompostion(df_production, area:str = 'NO1',group:str = 'hydro',period:int = 24,seasonal:int = 4*10+1,trend:int =24*30+1 ,robust:bool = True):
    df_subset = df_production[(df_production['pricearea']==area) & (df_production['productiongroup']==group)]
    df_subset.reset_index(inplace=True,drop=True)
    stl = STL(df_subset["quantitykwh"], period=period,seasonal=seasonal,trend=trend,robust=True)
    res = stl.fit() # Contains the components and a plot function
    ## Plot the results
    fig = go.Figure()
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True,subplot_titles=["Observed", "Trend", "Seasonal", "Residual"],vertical_spacing=0.05)
    # original data
    fig.add_trace(go.Scatter(x=df_subset['starttime'],y=df_subset["quantitykwh"],mode='lines',name='Observed',line=dict(color='blue')),row=1, col=1)
    # Trend
    fig.add_trace(go.Scatter(x=df_subset['starttime'],y=res.trend,mode='lines',name='Trend',line=dict(color='orange')),row=2, col=1)
    # Seasonal
    fig.add_trace(go.Scatter(x=df_subset['starttime'],y=res.seasonal,mode='lines',name='Seasonal',line=dict(color='green')),row=3, col=1)
    # Residual
    fig.add_trace(go.Scatter(x=df_subset['starttime'],y=res.resid,mode='lines',name='Residual',line=dict(color='gray', dash='dot')),row=4, col=1)
    fig.update_layout(title=f'The STL decomposition of area:{area} and productiongroup:{group}', xaxis4_title='Time (hourly)', yaxis_title='Values',height=900)

    return fig

In [ ]:
fig = plot_stl_decompostion(df_production)
fig.show()

### 5.Spectrogram

In [157]:
def plot_spectrogram_elhub(df_production,area: str = "NO1",group: str = "hydro",nperseg: int = 40,noverlap: int = 20):
    df_subset = df_production[(df_production["pricearea"] == area)& (df_production["productiongroup"] == group)].sort_values("starttime")
    y = df_subset["quantitykwh"].values
    fs = 1
    f, t, Zxx = stft(y, fs=fs, nperseg=nperseg, noverlap=noverlap)

    fig = go.Figure()
    magnitude = np.abs(Zxx)
    start_time = df_subset["starttime"].iloc[0]
    t_datetime = [start_time + pd.Timedelta(hours=float(h)) for h in t]

    fig.add_trace(go.Heatmap(
        x=t_datetime,
        y=f,
        z=magnitude,
        colorscale="Viridis",
        colorbar=dict(title="Amplitude"),
        zmin=0,
        zmax=magnitude.max() * 0.8,
        hovertemplate="Date: %{x|%Y-%m-%d %H:%M}<br>Freq: %{y:.4f}/h<br>Amp: %{z:.2f}<extra></extra>"
    ))

    fig.update_layout(
        title=f"Spectrogram of {group} production — {area}",
        xaxis_title="Date",
        yaxis_title="Frequency [1/hour]",
        template="plotly_white",
        height=600,
    )
    fig.update_yaxes(range=[0, 0.05])
    return fig

In [158]:
fig = plot_spectrogram_elhub(df_production)
fig.show()

Create a spectrogram based on the production data from elhub.
- Let the electricity price area, production group, window length and window overlap be parameters, and give each of them sensible defaults.
- Wrap this in a function that returns the plot, and test the function.